In [3]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from feature_engine.outliers import Winsorizer
import joblib

In [4]:
dataset = pd.read_csv("drug_regulatory_Classification_dataset.csv")

In [5]:
dataset.head()

,Dosage_mg,Price_Per_Unit,Production_Cost,Marketing_Spend,Clinical_Trial_Phase,Side_Effect_Severity_Score,Abuse_Potential_Score,Prescription_Rate,Hospital_Distribution_Percentage,Pharmacy_Distribution_Percentage,...,Manufacturing_Region,Requires_Cold_Storage,OTC_Flag,High_Risk_Substance,Insurance_Coverage_Percentage,Export_Percentage,Online_Sales_Percentage,Brand_Reputation_Score,Doctor_Recommendation_Rate,Target_Regulatory_Class
0,250,364.22,246.49,159132.53,3,4.26,2.52,0.74,57.93,42.07,...,South,No,No,No,64.46,41.88,41.15,6.56,0.47,Non-Regulated Drug
1,500,112.86,73.22,260595.45,2,8.12,1.80,0.48,48.00,52.00,...,East,Yes,Yes,No,93.37,10.30,50.05,4.01,0.44,Non-Regulated Drug
2,200,197.24,100.72,106818.45,3,1.58,1.47,0.93,54.48,45.52,...,North,Yes,Yes,No,64.53,34.21,46.42,5.95,0.16,Non-Regulated Drug
3,500,373.55,264.76,231304.59,4,1.72,1.61,0.54,47.20,52.80,...,North,No,Yes,No,95.21,71.04,49.52,9.65,0.76,Non-Regulated Drug
4,500,353.87,277.29,319403.02,2,9.64,0.45,0.50,86.96,13.04,...,North,No,Yes,No,98.05,70.28,44.90,1.79,0.36,Non-Regulated Drug


In [6]:
# separate features and target
x = dataset.iloc[:,1:]#features
y = dataset.iloc[:,0] # target

In [7]:
x.head()

,Price_Per_Unit,Production_Cost,Marketing_Spend,Clinical_Trial_Phase,Side_Effect_Severity_Score,Abuse_Potential_Score,Prescription_Rate,Hospital_Distribution_Percentage,Pharmacy_Distribution_Percentage,Annual_Sales_Volume,...,Manufacturing_Region,Requires_Cold_Storage,OTC_Flag,High_Risk_Substance,Insurance_Coverage_Percentage,Export_Percentage,Online_Sales_Percentage,Brand_Reputation_Score,Doctor_Recommendation_Rate,Target_Regulatory_Class
0,364.22,246.49,159132.53,3,4.26,2.52,0.74,57.93,42.07,32204.94,...,South,No,No,No,64.46,41.88,41.15,6.56,0.47,Non-Regulated Drug
1,112.86,73.22,260595.45,2,8.12,1.80,0.48,48.00,52.00,50963.01,...,East,Yes,Yes,No,93.37,10.30,50.05,4.01,0.44,Non-Regulated Drug
2,197.24,100.72,106818.45,3,1.58,1.47,0.93,54.48,45.52,4747.46,...,North,Yes,Yes,No,64.53,34.21,46.42,5.95,0.16,Non-Regulated Drug
3,373.55,264.76,231304.59,4,1.72,1.61,0.54,47.20,52.80,27985.78,...,North,No,Yes,No,95.21,71.04,49.52,9.65,0.76,Non-Regulated Drug
4,353.87,277.29,319403.02,2,9.64,0.45,0.50,86.96,13.04,23756.50,...,North,No,Yes,No,98.05,70.28,44.90,1.79,0.36,Non-Regulated Drug


In [8]:
y.head()

0    250
1    500
2    200
3    500
4    500
Name: Dosage_mg, dtype: int64

In [9]:
x.describe()

,Price_Per_Unit,Production_Cost,Marketing_Spend,Clinical_Trial_Phase,Side_Effect_Severity_Score,Abuse_Potential_Score,Prescription_Rate,Hospital_Distribution_Percentage,Pharmacy_Distribution_Percentage,Annual_Sales_Volume,...,Patent_Duration_Years,R&D_Investment_Million,Competitor_Count,Recall_History_Count,Adverse_Event_Reports,Insurance_Coverage_Percentage,Export_Percentage,Online_Sales_Percentage,Brand_Reputation_Score,Doctor_Recommendation_Rate
count,60000.000000,60000.000000,6.000000e+04,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,6.000000e+04,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000
mean,253.312360,151.954054,2.669683e+05,2.502750,5.498936,2.847217,0.526119,47.411735,52.588265,3.221264e+04,...,9.986983,100.536433,11.968967,0.499533,4.996517,59.793295,39.993931,29.900791,5.476783,0.525048
std,142.948109,92.028388,1.831812e+05,1.118055,2.604378,1.592573,0.245659,24.529169,24.529169,3.694159e+04,...,5.459745,57.332576,7.223363,0.708925,2.236252,23.046536,23.107853,17.313127,2.597910,0.244788
min,5.020000,2.070000,1.000535e+04,1.000000,1.000000,0.010000,0.100000,5.000000,10.000000,8.463400e+02,...,1.000000,1.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.010000,1.000000,0.100000
25%,129.790000,75.087500,1.335899e+05,2.000000,3.230000,1.610000,0.310000,26.150000,31.480000,1.303514e+04,...,5.000000,51.070000,6.000000,0.000000,3.000000,39.740000,19.900000,14.860000,3.230000,0.310000
50%,252.870000,146.395000,2.583855e+05,3.000000,5.500000,2.630000,0.530000,47.450000,52.550000,2.237015e+04,...,10.000000,100.750000,12.000000,0.000000,5.000000,59.740000,39.990000,29.990000,5.470000,0.530000
75%,377.572500,218.752500,3.819740e+05,4.000000,7.750000,3.880000,0.740000,68.520000,73.850000,3.875312e+04,...,15.000000,150.170000,18.000000,1.000000,6.000000,79.680000,59.950000,44.800000,7.720000,0.740000
max,499.960000,398.290000,1.999699e+06,4.000000,10.000000,9.350000,0.950000,90.000000,95.000000,1.248780e+06,...,19.000000,200.000000,24.000000,6.000000,16.000000,100.000000,80.000000,60.000000,10.000000,0.950000


In [10]:
y.describe()

count    60000.000000
mean       292.434167
std        214.975937
min         50.000000
25%        100.000000
50%        250.000000
75%        500.000000
max        650.000000
Name: Dosage_mg, dtype: float64

In [11]:
x.tail()

,Price_Per_Unit,Production_Cost,Marketing_Spend,Clinical_Trial_Phase,Side_Effect_Severity_Score,Abuse_Potential_Score,Prescription_Rate,Hospital_Distribution_Percentage,Pharmacy_Distribution_Percentage,Annual_Sales_Volume,...,Manufacturing_Region,Requires_Cold_Storage,OTC_Flag,High_Risk_Substance,Insurance_Coverage_Percentage,Export_Percentage,Online_Sales_Percentage,Brand_Reputation_Score,Doctor_Recommendation_Rate,Target_Regulatory_Class
59995,260.68,167.61,337160.40,2,6.04,2.79,0.92,67.01,32.99,16642.53,...,East,Yes,No,No,60.31,73.67,25.44,2.97,0.95,Regulated Drug
59996,249.67,139.19,457366.23,2,8.51,1.94,0.86,50.66,49.34,28399.80,...,South,Yes,No,No,31.87,38.08,56.10,6.75,0.91,Regulated Drug
59997,155.15,78.35,367705.08,4,6.45,2.36,0.85,26.03,73.97,10104.83,...,South,Yes,Yes,No,35.96,19.06,25.10,9.75,0.60,Non-Regulated Drug
59998,42.46,19.05,259954.00,1,6.98,2.29,0.16,39.96,60.04,11513.07,...,North,No,Yes,No,67.17,57.80,28.23,3.00,0.39,Regulated Drug
59999,153.99,83.04,267876.28,1,1.51,1.15,0.51,52.75,47.25,18881.65,...,North,Yes,Yes,No,82.20,10.26,22.85,5.38,0.87,Non-Regulated Drug


In [12]:
y.tail()

59995    200
59996    200
59997    100
59998    500
59999    250
Name: Dosage_mg, dtype: int64

In [13]:
categorical_features = x.select_dtypes(include=['object']).columns
numeric_features = x.select_dtypes(exclude=['object']).columns

In [14]:

# Numerical pipeline
num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('winsorizer', Winsorizer(capping_method='iqr', tail='both', fold=1.5)),
    ('scale', MinMaxScaler())
])

# Categorical pipeline
categ_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(drop='first'))
])

In [15]:
preprocess_pipeline = ColumnTransformer([
    ('numerical',num_pipeline,numeric_features),
    ('categorical',categ_pipeline,categorical_features)
])

In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Dosage_mg                         60000 non-null  int64  
 1   Price_Per_Unit                    60000 non-null  float64
 2   Production_Cost                   60000 non-null  float64
 3   Marketing_Spend                   60000 non-null  float64
 4   Clinical_Trial_Phase              60000 non-null  int64  
 5   Side_Effect_Severity_Score        60000 non-null  float64
 6   Abuse_Potential_Score             60000 non-null  float64
 7   Prescription_Rate                 60000 non-null  float64
 8   Hospital_Distribution_Percentage  60000 non-null  float64
 9   Pharmacy_Distribution_Percentage  60000 non-null  float64
 10  Annual_Sales_Volume               60000 non-null  float64
 11  Regulatory_Risk_Score             60000 non-null  float64
 12  Appr